In [5]:
import torch
from torch.autograd import Variable as V
import torchvision.models as models
from torchvision import transforms as trn
from torch.nn import functional as F
from torchvision import datasets

import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

from scipy.io import savemat
from tqdm import tqdm, trange

In [6]:
# the architecture to use
arch = 'fcn_resnet50'


In [7]:
def initialize_model(model_name, use_pretrained=True):
    model = None

    if model_name == "fcn_resnet50":
        model = models.segmentation.fcn_resnet50(pretrained=use_pretrained)

    else:
        print("Invalid model name, exiting...")
        exit()
    
    return model

In [8]:
model = initialize_model(arch)
model.eval()

FCN(
  (backbone): IntermediateLayerGetter(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequenti

In [5]:
## https://kozodoi.me/python/deep%20learning/pytorch/tutorial/2021/05/27/extracting-features.html

##### HELPER FUNCTION FOR FEATURE EXTRACTION

def get_features(name):
    def hook(model, input, output):
        features[name] = output.detach()
    return hook

##### REGISTER HOOK
if arch == "fcn_resnet50":
    model.backbone['conv1'].register_forward_hook(get_features('b_conv1'))
    model.backbone.layer1[2].conv3.register_forward_hook(get_features('b_conv2'))
    model.backbone.layer2[2].conv3.register_forward_hook(get_features('b_conv3'))
    model.backbone.layer3[2].conv3.register_forward_hook(get_features('b_conv4'))
    model.backbone.layer4[2].conv3.register_forward_hook(get_features('b_conv5'))
    model.classifier[4].register_forward_hook(get_features('c_conv'))
    model.aux_classifier[4].register_forward_hook(get_features('aux_conv'))

## ### Test the HYPOTHESIS

In [6]:
main_dir = "C:/Users/ASUS/Desktop/Object vision group - Stefania Bracci/Workplaces/Workplace_11042022/Python_extractFeatures"
data_dir = f"{main_dir}/data"

In [7]:
batch_size = 8

In [8]:
data_transforms = {
    'RSA': trn.Compose([
        trn.Resize((256,256)),
        trn.CenterCrop(224),
        trn.ToTensor(),
        trn.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

print("Initializing Datasets and Dataloaders for RSA...")

# Create training and validation datasets
our_image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x).replace("\\","/"), data_transforms[x]) for x in ['RSA']}
# Create training and validation dataloaders
our_dataloaders_dict = {x: torch.utils.data.DataLoader(our_image_datasets[x], batch_size=batch_size, shuffle=False, num_workers=2) for x in ['RSA']}

# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Initializing Datasets and Dataloaders for RSA...


In [9]:
## Save the names of images

# for 'RSA' set
nameTmp_dir_objects = f"{main_dir}/data/RSA\\1-objects\\"
len_dir_objects = len(nameTmp_dir_objects)
nameTmp_dir_both = f"{main_dir}/data/RSA\\2-both\\"
len_dir_both = len(nameTmp_dir_both)
nameTmp_dir_scenes = f"{main_dir}/data/RSA\\3-scenes\\"
len_dir_scenes = len(nameTmp_dir_scenes)


RSA_namesImg = []
N = len(our_image_datasets['RSA'].imgs)
for ii in range(0,N):
    nameTmp = our_image_datasets['RSA'].imgs[ii][0]
    if ii<64:
        RSA_namesImg.append(nameTmp[len_dir_objects:-4])
    elif ii<128:
        RSA_namesImg.append(nameTmp[len_dir_both:-4])
    elif ii<192:
        RSA_namesImg.append(nameTmp[len_dir_scenes:-4])

    
# RSA_namesImg 

In [10]:
classNames = our_dataloaders_dict['RSA'].dataset.classes
classNames, len(our_dataloaders_dict['RSA'].dataset)

(['1-objects', '2-both', '3-scenes'], 192)

## # Extract Features

In [11]:
##### FEATURE EXTRACTION LOOP
if arch == "fcn_resnet50":
    typeData = 'RSA' # for reference

    # placeholders
    b_conv1 = []
    b_conv2 = []
    b_conv3 = []
    b_conv4 = []
    b_conv5 = []

    c_conv = []
    aux_conv = []

    # placeholder for batch features
    features = {}

    for inputs, labels in tqdm(our_dataloaders_dict[typeData]):

        # forward pass
        preds = model(inputs)

        # add feats and preds to lists
        b_conv1.append(features['b_conv1'].numpy())
        b_conv2.append(features['b_conv2'].numpy())
        b_conv3.append(features['b_conv3'].numpy())
        b_conv4.append(features['b_conv4'].numpy())
        b_conv5.append(features['b_conv5'].numpy())

        c_conv.append(features['c_conv'].numpy())
        aux_conv.append(features['aux_conv'].numpy())


100%|██████████████████████████████████████████████████████████████████████████████████| 24/24 [02:45<00:00,  6.90s/it]


In [12]:
##### INSPECT FEATURES

b_conv1 = np.concatenate(b_conv1)
b_conv2 = np.concatenate(b_conv2)
b_conv3 = np.concatenate(b_conv3)
b_conv4 = np.concatenate(b_conv4)
b_conv5 = np.concatenate(b_conv5)

c_conv = np.concatenate(c_conv)
aux_conv = np.concatenate(aux_conv)

print('- b_conv1 shape:', b_conv1.shape)
print('- b_conv2 shape:', b_conv2.shape)
print('- b_conv3 shape:', b_conv3.shape)
print('- b_conv4 shape:', b_conv4.shape)
print('- b_conv5 shape:', b_conv5.shape)

print('- c_conv shape:', c_conv.shape)
print('- aux_conv shape:', aux_conv.shape)

- b_conv1 shape: (192, 64, 112, 112)
- b_conv2 shape: (192, 256, 56, 56)
- b_conv3 shape: (192, 512, 28, 28)
- b_conv4 shape: (192, 1024, 28, 28)
- b_conv5 shape: (192, 2048, 28, 28)
- c_conv shape: (192, 21, 28, 28)
- aux_conv shape: (192, 21, 28, 28)


In [13]:
resultDic = dict

if arch == "fcn_resnet50":
    resultDic = {"dataType": typeData, "RSA_namesImg": RSA_namesImg,
                 "b_conv1": b_conv1, "b_conv2": b_conv2, "b_conv3": b_conv3, "b_conv4": b_conv4, "b_conv5": b_conv5,
                 "c_conv": c_conv, "aux_conv": aux_conv}

save_dir = f"{main_dir}/resultDic/"
os.makedirs(save_dir, exist_ok = True)
savemat(f'{save_dir}{typeData}_resultDic_COCOpascal_{arch}.mat', resultDic)

In [ ]:
conv1[55,0,0,:]